In [1]:
import json
import numpy as np
import utils
import os

In [2]:
os.environ['PT_DATA_DIR'] = '/home/hoyeung/blob_matterport3d/'

In [3]:
with open('/home/hoyeung/blob_matterport3d/pretrain/asknav_pretrain_lookup.json', 'r') as f:
    pretrain_data = json.load(f)

In [43]:
len(pretrain_data)

139757

In [21]:
pretrain_data[0]

{'agent_path': [['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 13],
  ['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 14],
  ['7e78414b83ee4a378dfdcc91c052be65', 14],
  ['ee34e33014c94bf79cda5892e2ec6b24', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14]],
 'instr_id': '193593_0',
 'instruction': 'find pillows in one of the living rooms .',
 'scan': 'kEZ7cmS4wCh',
 'trajectory': [[0, 0, 0],
  [0, 1, 0],
  [1, 0, 0],
  [1, 0, 0],
  [2, 0, 0],
  [0, 0, 0]]}

In [6]:
scans = set([d['scan'] for d in pretrain_data])
len(scans)

56

In [7]:
scan_list = list(scans)

In [8]:
# Pick environment of 3 sizes
scan_size_map = {}
for scan in scans:
    scan_size_map[scan] = len(utils.load_nav_graphs(scan))

scan_size_map_items = scan_size_map.items()
sorted(scan_size_map_items, key=lambda x: x[1])

[('GdvgFV5R1Z5', 12),
 ('Pm6F8kyY3z2', 34),
 ('HxpKQynjfin', 34),
 ('17DRP5sb8fy', 44),
 ('jh4fc5c5qoQ', 48),
 ('JF19kD82Mey', 49),
 ('s8pcmisQ38h', 50),
 ('i5noydFURQK', 56),
 ('759xd9YjKW5', 61),
 ('VVfe2KiqLaN', 61),
 ('r47D5H71a5s', 61),
 ('EDJbREhghzL', 70),
 ('aayBHfsNo7d', 71),
 ('ZMojNkEp431', 77),
 ('JmbYfDe2QKZ', 79),
 ('sT4fr6TAbpF', 81),
 ('JeFG25nYj2p', 84),
 ('e9zR4mvMWw7', 87),
 ('5q7pvUzZiYa', 89),
 ('cV4RVeZvu5T', 90),
 ('1pXnuDYAj8r', 95),
 ('8WUmhLawc2A', 95),
 ('ULsKaCPVFJR', 101),
 ('29hnd4uzFmX', 102),
 ('7y3sRwLe3Va', 106),
 ('XcA2TqTSSAj', 109),
 ('S9hNv5qa7GM', 110),
 ('r1Q1Z4BcV1o', 113),
 ('b8cTxDM8gDG', 114),
 ('ur6pFq6Qu1A', 118),
 ('rPc6DW4iMge', 118),
 ('VLzqgDo317F', 123),
 ('pRbA3pwrgk9', 123),
 ('sKLMLpTHeUy', 127),
 ('1LXtFkjw3qL', 131),
 ('qoiz87JEwZ2', 131),
 ('82sE5b5pLXE', 135),
 ('Uxmj2M2itWa', 137),
 ('5LpN3gDmAk7', 140),
 ('Vvot9Ly1tCj', 141),
 ('p5wJjkQkbXX', 152),
 ('V2XKFyX4ASd', 157),
 ('ac26ZMwG7aT', 165),
 ('VFuaQ6m2Qom', 167),
 ('2n8kARJ

## pick unseen scans

In [28]:
np.random.seed(40)
random_indices = np.random.choice(56, size=6)
random_indices
# array([ 6, 27,  7, 37, 50,  1])

array([ 6, 27,  7, 37, 50,  1])

In [26]:
unseen_scan_list = [scan_list[i] for i in random_indices]
unseen_scan_list

# "\n['S9hNv5qa7GM',\n 'vyrNrziPKCB',\n 's8pcmisQ38h',\n '5q7pvUzZiYa',\n '1pXnuDYAj8r',\n '1pXnuDYAj8r']\n"

"\n['S9hNv5qa7GM',\n 'vyrNrziPKCB',\n 's8pcmisQ38h',\n '5q7pvUzZiYa',\n '1pXnuDYAj8r',\n '1pXnuDYAj8r']\n"

In [29]:
[scan_size_map[unseen_scan] for unseen_scan in unseen_scan_list]
# [77, 289, 56, 141, 106, 131]

[77, 289, 56, 141, 106, 131]

## split instructions (unseen, seen_val, seen_train)

In [70]:
seen_data = [d for d in pretrain_data if not (d['scan'] in unseen_scan_list)]
unseen_data = [d for d in pretrain_data if d['scan'] in unseen_scan_list]
print(len(pretrain_data))
print(len(seen_data))
print(len(unseen_data))

#139757
#119757
#20000

139757
119757
20000


In [80]:
np.random.seed(40)
seen_val_indices = np.random.choice(len(seen_data), len(unseen_data), replace=False)
seen_val_indices
# array([ 98552,  40760,  28296, ...,  23304, 102090,   1267])

array([ 98552,  40760,  28296, ...,  23304, 102090,   1267])

In [81]:
seen_val_data = [seen_data[i] for i in seen_val_indices]
seen_val_data[0]

{'agent_path': [['d5363662598b42b5b72a401065028040', 12],
  ['d5363662598b42b5b72a401065028040', 13],
  ['d5363662598b42b5b72a401065028040', 14],
  ['468e68b506364540972f4c718ca8c752', 14],
  ['468e68b506364540972f4c718ca8c752', 15],
  ['a73871837379498eb16527c6b9c9aa20', 15],
  ['b5145233e4ab457ba0c31f9eb20307bf', 15],
  ['b5145233e4ab457ba0c31f9eb20307bf', 16],
  ['35fbae92aa284ba19652ccb31f4cff9c', 16],
  ['6f5e70ae957d4a95af44068aa4946744', 16],
  ['3b1e1e0fc3de4418b73673c25ca83431', 16],
  ['3b1e1e0fc3de4418b73673c25ca83431', 15],
  ['3fe15c0ca3734249b98f3465e612b8db', 15],
  ['d7e72b80531c417095989e69398cf1d8', 15],
  ['e06ec7ada8304843a634f6aa91c9165d', 15],
  ['527864a208f246dbb52fb55adcb4751f', 15],
  ['b6169f1b5e0a42489ed145131e9ae486', 15],
  ['77fa4b2db2cf45a0abff5833de1bc079', 15],
  ['3e1701bd29544e47b96d79d370933af5', 15],
  ['3e1701bd29544e47b96d79d370933af5', 14],
  ['9785829df8fc43df823f9a74557b1e64', 14],
  ['9785829df8fc43df823f9a74557b1e64', 14]],
 'instr_id': '252

In [82]:
len(seen_val_data)

20000

In [84]:
seen_tr_data = [seen_data[i] for i in range(len(seen_data)) if not (i in seen_val_indices)]
seen_tr_data[0]

{'agent_path': [['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 13],
  ['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 14],
  ['7e78414b83ee4a378dfdcc91c052be65', 14],
  ['ee34e33014c94bf79cda5892e2ec6b24', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14]],
 'instr_id': '193593_0',
 'instruction': 'find pillows in one of the living rooms .',
 'scan': 'kEZ7cmS4wCh',
 'trajectory': [[0, 0, 0],
  [0, 1, 0],
  [1, 0, 0],
  [1, 0, 0],
  [2, 0, 0],
  [0, 0, 0]]}

In [85]:
len(seen_tr_data)

99757

## generate windows and split pos for validation sets

In [86]:
# take a look at unseen trajectory lengths
total_lens_unseen = [len(d['trajectory']) for d in unseen_data]
Counter(total_lens_unseen).most_common()

[(15, 1255),
 (17, 1250),
 (14, 1210),
 (13, 1179),
 (12, 1159),
 (16, 1150),
 (18, 1142),
 (19, 1120),
 (20, 1055),
 (11, 1024),
 (21, 989),
 (22, 966),
 (23, 961),
 (10, 920),
 (24, 917),
 (25, 857),
 (9, 850),
 (26, 716),
 (8, 616),
 (7, 393),
 (6, 271)]

In [87]:
# take a look at seen trajectory lengths
total_lens_seen = [len(d['trajectory']) for d in seen_data]
Counter(total_lens_seen).most_common()

[(15, 7204),
 (16, 7143),
 (17, 7086),
 (18, 7018),
 (14, 6879),
 (13, 6802),
 (19, 6801),
 (20, 6684),
 (12, 6489),
 (21, 6453),
 (22, 6108),
 (11, 6058),
 (23, 5756),
 (10, 5480),
 (24, 5365),
 (25, 4961),
 (9, 4849),
 (26, 4373),
 (8, 3792),
 (7, 2677),
 (6, 1779)]

In [103]:
# Set window size as 8

# sample swap pos
def sample_swap_pos(d):
    traj_len = len(d['trajectory'])
    swap_pos = np.random.choice(min(traj_len-1, 8-1))
    return swap_pos

def sample_window_pos(d):
    traj_len = len(d['trajectory'])
    if traj_len <= 8:
        return (0, traj_len-1)
    else:
        start_pos = np.random.choice(traj_len-8+1)
        end_pos = start_pos + 8 - 1
        return (start_pos, end_pos)
    
def sample_frame(split_data):
    for (i,d) in enumerate(split_data):
        split_data[i]['win_start_pos'], split_data[i]['win_end_pos'] = sample_window_pos(d)
        swap_bool = np.random.binomial(1, 0.5)
        if swap_bool:
            split_data[i]['swap_pos'] = sample_swap_pos(d)
        else:
            split_data[i]['swap_pos'] = None
    return split_data

In [107]:
# only do this for validation datasets
unseen_data_windowed = sample_frame(unseen_data)
seen_val_data_windowed = sample_frame(seen_val_data)

In [108]:
unseen_data_windowed[0]

{'agent_path': [['27b26e3f29384e60944fc6151074b172', 12],
  ['27b26e3f29384e60944fc6151074b172', 13],
  ['dd9fb6ab085d4d558d5d023bb2dc90ff', 13],
  ['dd9fb6ab085d4d558d5d023bb2dc90ff', 14],
  ['1d5c1b5e0a164fd8860723b2069f9b6a', 14],
  ['1d5c1b5e0a164fd8860723b2069f9b6a', 14]],
 'instr_id': '36151_0',
 'instruction': 'find a picture in the entryway or foyer or lobby .',
 'scan': '7y3sRwLe3Va',
 'swap_pos': 3,
 'trajectory': [[0, 0, 0],
  [0, 1, 0],
  [1, 0, 0],
  [0, 1, 0],
  [2, 0, 0],
  [0, 0, 0]],
 'win_end_pos': 5,
 'win_start_pos': 0}

In [110]:
seen_val_data_windowed[0]

{'agent_path': [['d5363662598b42b5b72a401065028040', 12],
  ['d5363662598b42b5b72a401065028040', 13],
  ['d5363662598b42b5b72a401065028040', 14],
  ['468e68b506364540972f4c718ca8c752', 14],
  ['468e68b506364540972f4c718ca8c752', 15],
  ['a73871837379498eb16527c6b9c9aa20', 15],
  ['b5145233e4ab457ba0c31f9eb20307bf', 15],
  ['b5145233e4ab457ba0c31f9eb20307bf', 16],
  ['35fbae92aa284ba19652ccb31f4cff9c', 16],
  ['6f5e70ae957d4a95af44068aa4946744', 16],
  ['3b1e1e0fc3de4418b73673c25ca83431', 16],
  ['3b1e1e0fc3de4418b73673c25ca83431', 15],
  ['3fe15c0ca3734249b98f3465e612b8db', 15],
  ['d7e72b80531c417095989e69398cf1d8', 15],
  ['e06ec7ada8304843a634f6aa91c9165d', 15],
  ['527864a208f246dbb52fb55adcb4751f', 15],
  ['b6169f1b5e0a42489ed145131e9ae486', 15],
  ['77fa4b2db2cf45a0abff5833de1bc079', 15],
  ['3e1701bd29544e47b96d79d370933af5', 15],
  ['3e1701bd29544e47b96d79d370933af5', 14],
  ['9785829df8fc43df823f9a74557b1e64', 14],
  ['9785829df8fc43df823f9a74557b1e64', 14]],
 'instr_id': '252

In [111]:
seen_val_data_windowed[1]

{'agent_path': [['895dc1e515e6491abd794cfdda27edd3', 13],
  ['d470265ca129400bb48754d16a157f9e', 13],
  ['e6040538e6ec43a7b73be5b9614ebd68', 13],
  ['9940d688fe8f4fb784cd7d64908fdf4d', 13],
  ['fa065de5315940abb64684e68f6d686a', 13],
  ['fa065de5315940abb64684e68f6d686a', 12],
  ['fa065de5315940abb64684e68f6d686a', 23],
  ['fa065de5315940abb64684e68f6d686a', 22],
  ['02253cded39e4f26a34c483d645ebcb7', 22],
  ['02253cded39e4f26a34c483d645ebcb7', 21],
  ['ec0421cc61c64f6f8692d327d6838fd5', 21],
  ['3510eb416b7d4d779881abc5b5cf2cec', 21],
  ['e3b1895ff0cb41b19be7fb13aa4e8d9a', 21],
  ['e3b1895ff0cb41b19be7fb13aa4e8d9a', 21]],
 'instr_id': '143331_1',
 'instruction': 'find a bed in one of the bedrooms .',
 'scan': 'XcA2TqTSSAj',
 'swap_pos': None,
 'trajectory': [[0, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [2, 0, 0],
  [0, -1, 0],
  [0, -1, 0],
  [0, -1, 0],
  [1, 0, 0],
  [0, -1, 0],
  [1, 0, 0],
  [1, 0, 0],
  [3, 0, 0],
  [0, 0, 0]],
 'win_end_pos': 12,
 'win_start_pos': 5}

In [112]:
seen_tr_data[0]

{'agent_path': [['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 13],
  ['50ffb35ba8654fd1a6a3cbdd4e5a29b7', 14],
  ['7e78414b83ee4a378dfdcc91c052be65', 14],
  ['ee34e33014c94bf79cda5892e2ec6b24', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14],
  ['bc28e77e30a94f76aee87ac07949ecc4', 14]],
 'instr_id': '193593_0',
 'instruction': 'find pillows in one of the living rooms .',
 'scan': 'kEZ7cmS4wCh',
 'trajectory': [[0, 0, 0],
  [0, 1, 0],
  [1, 0, 0],
  [1, 0, 0],
  [2, 0, 0],
  [0, 0, 0]]}

## Save the datasets

In [113]:
with open('/home/hoyeung/blob_matterport3d/pretrain/asknav_pretrain_lookup_train.json', 'w') as f:
    json.dump(seen_tr_data, f)

In [114]:
with open('/home/hoyeung/blob_matterport3d/pretrain/asknav_pretrain_lookup_val_seen.json', 'w') as f:
    json.dump(seen_val_data, f)

In [115]:
with open('/home/hoyeung/blob_matterport3d/pretrain/asknav_pretrain_lookup_val_unseen.json', 'w') as f:
    json.dump(unseen_data, f)

In [117]:
lis = [(1,2,3),(4,5,6),(7,8,9)]
lis.insert(0, lis[0])
lis

[(1, 2, 3), (1, 2, 3), (4, 5, 6), (7, 8, 9)]

In [118]:
lis.insert(-1, lis[-1])
lis

[(1, 2, 3), (1, 2, 3), (4, 5, 6), (7, 8, 9), (7, 8, 9)]

In [119]:
0 % 2

0

In [120]:
1 % 2

1

In [121]:
3 % 2

1

In [122]:
2 % 1

0